In [1]:
import numpy as np
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD
from torch.nn import CrossEntropyLoss, MSELoss
from pennylane import NesterovMomentumOptimizer
from thesis.qcnn import QCNN

dims = (16, 16), (28, 28), (32, 32)
num_trials = 1

In [2]:
from thesis.operation.ansatz import BaselineAnsatz
from thesis.ml.data import Data
from thesis.ml.optimize import Optimizer
from thesis.ml.data import image_transform

data = Data(FashionMNIST, image_transform((16, 16)), None, classes=[0, 1], batch_size=40)
optimizer = Optimizer(SGD, lr=0.01, momentum=0.9, nesterov=True)
qcnn = QCNN(data, optimizer, CrossEntropyLoss())

for _ in range(num_trials):
    accuracy = qcnn(BaselineAnsatz, (16, 16))
    print(f"New ansatz: {accuracy=:.3%}")

51


TypeError: object of type 'NoneType' has no len()